In [4]:
pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 2.6 MB/s eta 0:00:00
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-3.1.0-cp39-cp39-macosx_10_9_x86_64.whl (124 kB)
  Using cached urllib3-2.0.2-py3-none-any.whl (123 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 5.3 MB/s eta 0:00:006.3 MB/s eta 0:00:01
  Using cached numpy-1.24.3-cp39-cp39-macosx_10_9_x86_64.whl (19.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 KB 5.7 MB/s eta 0:00:00m eta 0:00:010:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [64]:


import requests
import json
import pandas as pd
retList = []


In [65]:

#호출용 파라미터 지정
def getReviewParam(place, theme, pageNum = 1):
    
    
    return {
    "operationName": "getVisitorReviews",
    "variables": {
      "input": {
        "businessId": place['id'],
        "businessType": "restaurant",
        "item": "0",
        "page": pageNum,
        "size": 40,
        "includeContent": True,
        "getUserStats": True,
        "includeReceiptPhotos": True,
        "cidList": place['categoryPath'][0],
        "theme": theme
      },
      "id": place['id']
    },
  "query": "query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        class\n        videoId\n        videoOriginSource\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        replyTitle\n        isReported\n        isSuspended\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      language\n      highlightOffsets\n      apolloCacheId\n      translatedText\n      businessName\n      showBookingItemName\n      showBookingItemOptions\n      bookingItemName\n      bookingItemOptions\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      userIdno\n      isFollowing\n      followerCount\n      followRequested\n      loginIdno\n      receiptInfoUrl\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    showRecommendationSort\n    itemReviewStats {\n      score\n      count\n      itemId\n      starDistribution {\n        score\n        count\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"
  }



#카테고리 현황 조회용 파라미터
def getStatParam(place):
    return {
  "operationName": "getVisitorReviewStats",
  "variables": {
    "businessType": "restaurant",
    "id": place['id'],
    "itemId": "0"
  },
  "query": "query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n"
}



# {
#   "operationName": "getVisitorReviewStats",
#   "variables": {
#     "businessType": "restaurant",
#     "id": "1813392509",
#     "itemId": "0"
#   },
#   "query": "query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n"
# }




def placeProc(place):
    #장소별 ret 초기화
    placeDict = {}

    placeDict['id'] =  place['id']
    placeDict['name'] =  place['name']
    placeDict['contents'] =  []
    
    statParam = getStatParam(place)

    # 리뷰/테마를 읽어오기 위한 api 정의
    review_url = "https://pcmap-api.place.naver.com/graphql"
    review_headers = {
        "referer": "https://pcmap.place.naver.com/",
        "content-type": "application/json",
        "accept": "*/*",
        "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"
    }

    # 테마 목록 가져오기 
    stat_response = requests.post(review_url, headers=review_headers, data=json.dumps(statParam))
    print('stat request', place['name'], stat_response)
    if stat_response.status_code != 200:        
        return placeDict;
    stat_json = json.loads(stat_response.text)

    # 테막 목록 확인
    themes = stat_json['data']['visitorReviewStats']['analysis']['themes']
    # 테마별 리뷰를 불러온다.
    placeDict['contents'] = []
    for theme in themes:
        # total은 제외한다
        if theme['code'] in ['taste','service','price']:
            nextPage = True
            pageNum = 1
            print (pageNum, nextPage)
            while nextPage:
              print (pageNum, nextPage)
              review_param = getReviewParam(place,theme['code'], pageNum)
              review_response = requests.post(review_url, headers=review_headers, data=json.dumps(review_param))
              if review_response.status_code != 200: 
                  continue;
              review_json = json.loads(review_response.text)
              reviewBody = review_json['data']['visitorReviews']['items']
              print(theme['code'], len(reviewBody))
              placeDict['contents'].extend(list(map(lambda x: {'div': theme['code'], 'cont':x['body']} , reviewBody)))
              if len(reviewBody)==40:
                  pageNum = pageNum + 1
              else:
                  nextPage = False
                  break
            # #반환된 review 건수가 40건인 경우 page 2 한번 더 호출한다.
            # if len(reviewBody)==40:
            #     review_param = getReviewParam(place, theme['code'], 2)
            #     review_response = requests.post(review_url, headers=review_headers, data=json.dumps(review_param))
            #     if review_response.status_code != 200: 
            #         continue;
            #     review_json = json.loads(review_response.text)
            #     reviewBody = review_json['data']['visitorReviews']['items']
            #     print(theme['code'], len(reviewBody))
            #     placeDict['contents'].extend(list(map(lambda x: {'div': theme['code'], 'cont':x['body']} , reviewBody)))

    return placeDict
    
    

In [66]:
url = 'https://map.naver.com/v5/api/search'

headers = {
    'authority': 'map.naver.com',
    'content-type': 'application/json',
    "accept": "*/*",
    "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"
}

### 음식점 목록을 가져온다 (현재 위치 위경도값:searchCoord, query : 검색어)
### 호출건수 : displayCount : 100
params = {
    'lang': 'ko',
    'caller': 'pcweb',
    'type': 'place',
    #'searchCoord': '37.272429000000116,127.13596284980378', #용인
    'query': '솔솥 성수점',
    'page': 1,
    'displayCount': 1,
    'isPlaceRecommendationReplace': 'false',
}


# clientX: 127.1275729
# clientY: 37.272429
# x: 127.10589175393739
# y: 37.36958999999972
# bounds: 127.09966902902255;37.363723468708145;127.12069754769959;37.37528555368495


In [67]:

response = requests.get(url, headers=headers, params=params)

result = json.loads(response.text)

#호출결과에서 장소에 대한 목록만 읽어온다
placeList = result['result']['place']['list']

In [68]:
len(placeList)

1

In [69]:
print(placeList)

[{'index': '0', 'rank': '1', 'id': '1580152953', 'name': '솔솥 성수점', 'tel': '010-2046-1130', 'isCallLink': False, 'virtualTel': '0507-1339-2007', 'virtualTelDisplay': '0507-1339-2007', 'ppc': '1', 'category': ['음식점', '한식'], 'categoryPath': [['220036', '220037', '1005097'], ['1004760', '1004380', '1002043']], 'rcode': '09200115', 'businessStatus': {'requestTime': '202305302012', 'status': {'code': 2, 'text': '영업 중', 'emphasis': False, 'description': '영업중', 'detailInfo': '20:30에 라스트오더'}, 'businessHours': '202305301130~202305302100', 'breakTime': '202305301500~202305301700', 'lastOrder': '202305302030'}, 'naviInfoText': None, 'address': '서울특별시 성동구 성수동2가 315-25 솔솥 성수점', 'roadAddress': '서울특별시 성동구 연무장5가길 24 솔솥 성수점', 'abbrAddress': '성수동2가 315-25 솔솥 성수점', 'shortAddress': ['서울 성동구', '연무장5가길 24', '솔솥 성수점'], 'display': '<b>솔솥 성수점</b>', 'telDisplay': '010-2046-1130', 'context': [], 'reviewCount': 1057, 'placeReviewCount': 950, 'ktCallMd': '2af4516b73539ba328f4f52b62d3b28f', 'coupon': '0', 'thumUrl':

In [70]:
# 길이 확인 및 첫 음식점 확인
print(len(placeList))
retList = []
placeList[0]
print(placeList[0]['id'])
cnt = 0
for place in placeList:
    cnt+=1
    # if cnt<50: continue
    retList.append(placeProc(place))
    
    
# retList.append(placeProc(placeList[1]))
# retList.append(placeProc(placeList[2]))

1
1580152953
stat request 솔솥 성수점 <Response [200]>
1 True
1 True
1 True
taste 40
2 True
taste 40
3 True
3 True
3 True
3 True
taste 40
4 True
taste 40
5 True
5 True
taste 40
6 True
taste 40
7 True
7 True
7 True
7 True
taste 40
8 True
taste 40
9 True
9 True
taste 40
10 True
taste 21
1 True
1 True
service 40
2 True
service 32
1 True
1 True
price 15


In [71]:
retJson = json.dumps(retList)

from datetime import datetime
nowstr = datetime.now().strftime('%Y.%m.%d.%H%M%S') # 년.월.일 - 시간
with open("./retJson_{0}.json".format(nowstr), 'w', encoding='utf8') as outfile:
    json.dump(retList, outfile, ensure_ascii=False)

In [72]:
import os

# 폴더 경로
folder_path = "./"

# JSON 파일 리스트
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# JSON 파일 읽기 및 병합
merged_json = []
for file_name in json_files:
    print(file_name)
    with open(os.path.join(folder_path, file_name), 'r') as file:
        json_data = json.load(file)
        merged_json.extend(json_data[::-1])

nowstr = datetime.now().strftime('%Y.%m.%d.%H%M%S') # 년.월.일 - 시간

# 병합된 JSON 파일 저장
with open("./merged_{0}.json".format(nowstr), 'w', encoding='utf8') as file:
    json.dump(merged_json, file, ensure_ascii=False)

retJson_2023.05.30.201307.json


In [73]:
merged_json

[{'id': '1580152953',
  'name': '솔솥 성수점',
  'contents': [{'div': 'taste',
    'cont': '갈치솥밥, 스테이크솥밥, 꼬막솥밥, 도미관자솥밥, 유린기, 새우튀김 먹었어요.\n유린기와 새우튀김은 반찬으로 적절해요. 유린기는 추가로 주문하고 싶을만큼 맛있어요. 오픈런 했는데도 30분 기다려 먹었네요.\n밥이 촉촉하고 맛있어서 한 그릇 뚝딱 하게 됩니다. :) \n고모가 신부전증을 앓고 계셔서 소금 빼달라고 했는데 재료 본연의 맛을 느낄 수 있는 건강한 맛이었어요. 손님 모시고 가기 좋은 식당 같아요. 외국인들이 많이 가더라고요. '},
   {'div': 'taste',
    'cont': '스테이크덮밥 맛있습니다. \n고기 추가 한 건데 \n남자는 추가해야 양이 맞을 거 같아요. \n장어덮밥도 맛있긴한데 가격 생각하면  좀 아쉬웠습니다.\n새우튀김과 유린기 무난하게 괜찮았습니다.\n반찬 깔끔하고 분위기도 좋아 맛있게 잘 먹었습니다.\n카프리 시켰는데 안시원해서... 이건 많이 아쉬웠습니다.'},
   {'div': 'taste',
    'cont': '일본어로 씌여 있어 한 바퀴 더 돌아 찾아냈습니다😳\n출입문 바로 옆 전번을 입력하고 대기합니다~\n메뉴 주문까지 포함, 20분 정도 웨이팅 한 후 입장, 매우매우 친절하고 음식 맛도 나쁘지 않았습니다.\n\n재방문의사 있어요👍주차가능한 솔솥은 찾기 힘들어 더욱 좋으네요😎가게 앞 바로 주차 가능-협소하지만요^^-'},
   {'div': 'taste',
    'cont': '도미관자솥밥 16,000원 먹었어요!\n성수다보니 가격이 저렴하진 않지만 다른 식당들 생각하면 깔끔하고 합리적이라서 방문했어요.\n일단 스테이크보다 전 도미관자가 담백&고소해서 좋더라구요!\n진짜 생선 좋아하시는 분들 엄청 맛있게 드실 거 같네요. 특별할 거 없어보이는데 진짜 고소+슴슴 맛나요ㅋㅋ신기해요.\n김이랑 싸먹으면 극락...☆ 또 먹고싶네여\n주말이라 11팀 웨이팅있었

In [74]:
len(merged_json)

1

In [75]:
cnt = 0
last_json = []
for place in merged_json:
    if cnt>1: continue;
    if len(place['contents'])>0:
        # 'div' 키의 값이 'taste', 'service', 'price' 인 데이터만 저장할 리스트
        last_json.extend(list(filter(lambda x: x.get('div') in ['taste', 'service', 'price'], place['contents'])))

    

In [76]:
last_json

[{'div': 'taste',
  'cont': '갈치솥밥, 스테이크솥밥, 꼬막솥밥, 도미관자솥밥, 유린기, 새우튀김 먹었어요.\n유린기와 새우튀김은 반찬으로 적절해요. 유린기는 추가로 주문하고 싶을만큼 맛있어요. 오픈런 했는데도 30분 기다려 먹었네요.\n밥이 촉촉하고 맛있어서 한 그릇 뚝딱 하게 됩니다. :) \n고모가 신부전증을 앓고 계셔서 소금 빼달라고 했는데 재료 본연의 맛을 느낄 수 있는 건강한 맛이었어요. 손님 모시고 가기 좋은 식당 같아요. 외국인들이 많이 가더라고요. '},
 {'div': 'taste',
  'cont': '스테이크덮밥 맛있습니다. \n고기 추가 한 건데 \n남자는 추가해야 양이 맞을 거 같아요. \n장어덮밥도 맛있긴한데 가격 생각하면  좀 아쉬웠습니다.\n새우튀김과 유린기 무난하게 괜찮았습니다.\n반찬 깔끔하고 분위기도 좋아 맛있게 잘 먹었습니다.\n카프리 시켰는데 안시원해서... 이건 많이 아쉬웠습니다.'},
 {'div': 'taste',
  'cont': '일본어로 씌여 있어 한 바퀴 더 돌아 찾아냈습니다😳\n출입문 바로 옆 전번을 입력하고 대기합니다~\n메뉴 주문까지 포함, 20분 정도 웨이팅 한 후 입장, 매우매우 친절하고 음식 맛도 나쁘지 않았습니다.\n\n재방문의사 있어요👍주차가능한 솔솥은 찾기 힘들어 더욱 좋으네요😎가게 앞 바로 주차 가능-협소하지만요^^-'},
 {'div': 'taste',
  'cont': '도미관자솥밥 16,000원 먹었어요!\n성수다보니 가격이 저렴하진 않지만 다른 식당들 생각하면 깔끔하고 합리적이라서 방문했어요.\n일단 스테이크보다 전 도미관자가 담백&고소해서 좋더라구요!\n진짜 생선 좋아하시는 분들 엄청 맛있게 드실 거 같네요. 특별할 거 없어보이는데 진짜 고소+슴슴 맛나요ㅋㅋ신기해요.\n김이랑 싸먹으면 극락...☆ 또 먹고싶네여\n주말이라 11팀 웨이팅있었네요.. \n그래도 앞에 대기석있어서 수다 떠니 기다릴만 했어요.(테이블링 돼요)\n가게 인테리어도 깔끔하고 좋았습니당\n누룽

In [77]:
df = pd.DataFrame(last_json)

In [78]:
df

,div,cont
0,taste,"갈치솥밥, 스테이크솥밥, 꼬막솥밥, 도미관자솥밥, 유린기, 새우튀김 먹었어요.\n유..."
1,taste,스테이크덮밥 맛있습니다. \n고기 추가 한 건데 \n남자는 추가해야 양이 맞을 거 ...
2,taste,일본어로 씌여 있어 한 바퀴 더 돌아 찾아냈습니다😳\n출입문 바로 옆 전번을 입력하...
3,taste,"도미관자솥밥 16,000원 먹었어요!\n성수다보니 가격이 저렴하진 않지만 다른 식당..."
4,taste,솔솥 성수점 주말이라 웨이팅은 있었지만 솥밥 너무 맛있게 먹음. 맛이 자극적이지 않...
...,...,...
463,price,맛있는 한끼 좋아요~\n솥밥 좋아하는데 간도 짜지않고 적당해서 좋았어요 \n가격은 ...
464,price,가격대비 양이 적어요.\n남자라서 그런가??
465,price,도미관자솥밥 담백하고 깔끔하게 맛있어요 가격대가 좀 있긴하지만 재료가 좋으니깐 다음...
466,price,가격은 비싸지만 맛이 있네요. 단지 대기가 있는데 더운여름 밖에서 기다리긴 힘드네요


In [79]:
df.to_csv('solsot.csv')